In [28]:
import csv
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from sentence_transformers.readers import InputExample
from sentence_transformers import SentenceTransformer, losses
from scipy.stats import pearsonr

!pip 

In [11]:
import pandas as pd
df = pd.read_csv('dev.csv', sep='\t', on_bad_lines='skip')


0       5.00
1       4.75
2       5.00
3       2.40
4       2.75
        ... 
1465    2.00
1466    0.00
1467    2.00
1468    0.00
1469    0.00
Name: score, Length: 1470, dtype: float64

In [14]:
lst = []
for i in range(df.shape[0]):
    lst.append({'score':df['score'][i], 'sentence1':df['sentence1'][i], 'sentence2':df['sentence2'][i]})

In [31]:
class Dataset1b(Dataset):

    def __init__(self,data):
        self.s1 = [d['sentence1'] for d in data]
        self.s2 = [d['sentence2'] for d in data]
        self.score = [d['score'] for d in data]

    
    def __len__(self):
        return len(self.score)
    
    def __getitem__(self, index):
        
        cos_similarity = torch.tensor(self.score[index])

        return InputExample(texts=[self.s1[index],self.s2[index]] , label=cos_similarity)

        


In [24]:
dataset_1b = Dataset1b(lst)
dataloader_1b = DataLoader(dataset_1b , batch_size=16, shuffle=True)

In [27]:
model_1b = SentenceTransformer("all-MiniLM-L6-v2")
dataloader_1b.collate_fn = model_1b.smart_batching_collate

c:\Users\HITESH KUMAR\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HITESH KUMAR\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [35]:
pred = []
ground_truth= []
for i , batch in enumerate(dataloader_1b):
    embeddings = [model_1b(sentence_feature)["sentence_embedding"] for sentence_feature in batch[0]]
    cosine_similarity_pred = torch.cosine_similarity(embeddings[0], embeddings[1])
    
    score_pred = cosine_similarity_pred * 5
    score_gt = batch[1]
    pred.extend(score_pred.tolist())
    ground_truth.extend(score_gt.tolist())

print(f"The Pearson Correlation Coefficient is : {pearsonr(pred, ground_truth)[0]}")

The Pearson Correlation Coefficient is : 0.8611497882197772
